In [1]:
import pandas as pd
import datetime
import os
import json
from haversine import haversine

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
ratio_dist=25

In [2]:
pd.options.display.max_columns=99
input_locations=pd.read_csv("./testww.csv",dtype=str)
# lat lng given in the csv, just use it
# first place only
input_locations=input_locations.loc[0,['EVENT_ID','VENUE_LATITUDE','VENUE_LONGITUDE']].to_frame().T
input_locations['VENUE_LATITUDE']=input_locations['VENUE_LATITUDE'].astype(float)
input_locations['VENUE_LONGITUDE']=input_locations['VENUE_LONGITUDE'].astype(float)


In [3]:
df_TA=input_locations.copy()

df_store_zip=pd.DataFrame()

for ind,row in df_TA.iterrows():
    event_id=row['EVENT_ID']
    zip_list=[]
    store_zip=pd.DataFrame()
    store_center=df_TA[['VENUE_LATITUDE','VENUE_LONGITUDE']].values[0]
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_center,v,unit="mi") # unit="mi"
        if dist<=ratio_dist:
            zip_list.append(zip_cd)
            df=pd.DataFrame({"dist_miles":dist},index=[zip_cd])
            store_zip=store_zip.append(df)
    store_zip=store_zip.reset_index().rename(columns={"index":"zip_cd"})
    store_zip['EVENT_ID']=event_id
    df_TA.loc[ind,'zip_list']=str(zip_list)
    df_store_zip=df_store_zip.append(store_zip)

In [4]:
df_TA['zip_list']=df_TA['zip_list'].apply(eval)

In [5]:
df_store_zip=df_store_zip[['EVENT_ID','zip_cd','dist_miles']]

In [7]:
writer=pd.ExcelWriter("./Trade_area_1_event_25_miles_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_store_zip.to_excel(writer,"by_store_zip",index=False)
df_TA.to_excel(writer,"1_event",index=False)
writer.save()